In [10]:
import pygame
import sys
import random
import copy
from pygame.locals import *

# Initialize pygame
pygame.init()

# Constants
FPS = 60
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 800
GRID_SIZE = 80
GRID_WIDTH = 5
GRID_HEIGHT = 5

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BROWN = (174, 94, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
EMPTY = None

# Fonts
FONT = pygame.font.SysFont('Arial', 20)
BIG_FONT = pygame.font.SysFont('Arial', 30)

class Fanorona:
    def __init__(self):
        self.main_clock = pygame.time.Clock()
        self.window_surf = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption('Fanorona 5X5')
        
        # Game state
        self.difficulty = None
        self.human_token = None
        self.AI_token = None
        self.turn = None
        self.game_over = False
        self.winner = None
        self.selected_piece = None
        self.valid_moves = []
        
        # AI statistics
        self.AI_current_action = {}
        self.is_cutoff = False
        self.depth_of_game_tree = 0
        self.total_node_generated = 0
        self.pruning_in_max_value = 0
        self.pruning_in_min_value = 0
        
        # Initialize board
        self.main_grid = self.get_new_grid_5X5()
    
    def get_new_grid_5X5(self):
        """Create a new 5x5 grid with initial piece positions"""
        grid = [[EMPTY for _ in range(GRID_HEIGHT)] for _ in range(GRID_WIDTH)]
        
        # Set up initial pieces (White on top, Black on bottom)
        for col in range(GRID_WIDTH):
            for row in [0, 1]:  # First two rows
                grid[col][row] = WHITE
            for row in [3, 4]:  # Last two rows
                grid[col][row] = BLACK
        
        return grid
    
    def draw_grid(self):
        """Draw the game board and pieces"""
        self.window_surf.fill(BROWN)
        
        # Draw grid lines
        for col in range(GRID_WIDTH):
            for row in range(GRID_HEIGHT):
                center_x, center_y = self.translate_grid_to_pixel_coord(col, row)
                rect = pygame.Rect(center_x - GRID_SIZE//2, center_y - GRID_SIZE//2, GRID_SIZE, GRID_SIZE)
                pygame.draw.rect(self.window_surf, (200, 150, 100), rect, 1)
                
                # Draw pieces
                if self.main_grid[col][row] != EMPTY:
                    color = self.main_grid[col][row]
                    pygame.draw.circle(self.window_surf, color, (center_x, center_y), GRID_SIZE//2 - 5)
                    pygame.draw.circle(self.window_surf, BLACK, (center_x, center_y), GRID_SIZE//2 - 5, 2)
        
        # Highlight selected piece
        if self.selected_piece:
            col, row = self.selected_piece
            center_x, center_y = self.translate_grid_to_pixel_coord(col, row)
            pygame.draw.circle(self.window_surf, YELLOW, (center_x, center_y), GRID_SIZE//2 - 3, 3)
        
        # Highlight valid moves
        for (col, row) in self.valid_moves:
            center_x, center_y = self.translate_grid_to_pixel_coord(col, row)
            pygame.draw.circle(self.window_surf, GREEN, (center_x, center_y), 10)
        
        # Draw game info
        turn_text = FONT.render(f"Turn: {'Human' if self.turn == 'Human' else 'AI'}", True, BLACK)
        self.window_surf.blit(turn_text, (20, 20))
        
        diff_text = FONT.render(f"Difficulty: {self.difficulty}", True, BLACK)
        self.window_surf.blit(diff_text, (20, 50))
        
        # Draw AI stats when it's AI's turn
        if self.turn == 'AI':
            stats = [
                f"Max Depth: {self.depth_of_game_tree}",
                f"Nodes: {self.total_node_generated}",
                f"Pruning (max/min): {self.pruning_in_max_value}/{self.pruning_in_min_value}",
                f"Cutoff: {'Yes' if self.is_cutoff else 'No'}"
            ]
            for i, stat in enumerate(stats):
                stat_text = FONT.render(stat, True, BLACK)
                self.window_surf.blit(stat_text, (WINDOW_WIDTH - 220, 20 + i * 25))
        
        pygame.display.update()
    
    def translate_grid_to_pixel_coord(self, col, row):
        """Convert grid coordinates to pixel coordinates"""
        margin_x = (WINDOW_WIDTH - (GRID_WIDTH * GRID_SIZE)) // 2
        margin_y = (WINDOW_HEIGHT - (GRID_HEIGHT * GRID_SIZE)) // 2
        return (margin_x + col * GRID_SIZE + GRID_SIZE//2, 
                margin_y + row * GRID_SIZE + GRID_SIZE//2)
    
    def get_grid_clicked(self, mouse_x, mouse_y):
        """Return grid coordinates from mouse click position"""
        margin_x = (WINDOW_WIDTH - (GRID_WIDTH * GRID_SIZE)) // 2
        margin_y = (WINDOW_HEIGHT - (GRID_HEIGHT * GRID_SIZE)) // 2
        
        if (margin_x <= mouse_x < margin_x + GRID_WIDTH * GRID_SIZE and
            margin_y <= mouse_y < margin_y + GRID_HEIGHT * GRID_SIZE):
            col = (mouse_x - margin_x) // GRID_SIZE
            row = (mouse_y - margin_y) // GRID_SIZE
            return (col, row)
        return (None, None)
    
    def show_difficulty_menu(self):
        """Show difficulty selection menu"""
        self.window_surf.fill(BROWN)
        
        title = BIG_FONT.render("Welcome to Fanorona Player", True, BLACK)
        subtitle = FONT.render("Please choose your difficulty", True, BLACK)
        self.window_surf.blit(title, (WINDOW_WIDTH//2 - title.get_width()//2, 150))
        self.window_surf.blit(subtitle, (WINDOW_WIDTH//2 - subtitle.get_width()//2, 200))
        
        options = [
            "1 (random move)",
            "2 (one depth)",
            "3 (deep depth)"
        ]
        
        for i, option in enumerate(options):
            option_text = FONT.render(option, True, BLACK)
            self.window_surf.blit(option_text, (WINDOW_WIDTH//2 - option_text.get_width()//2, 300 + i * 50))
        
        pygame.display.flip()
        
        while True:
            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == KEYDOWN:
                    if event.key == K_1:
                        return '1'
                    elif event.key == K_2:
                        return '2'
                    elif event.key == K_3:
                        return '3'
            self.main_clock.tick(FPS)
    
    def show_color_menu(self):
        """Let player choose their token color"""
        self.window_surf.fill(BROWN)
        
        title = BIG_FONT.render("Choose your color", True, BLACK)
        self.window_surf.blit(title, (WINDOW_WIDTH//2 - title.get_width()//2, 150))
        
        white_text = FONT.render("Press W for White (moves first)", True, BLACK)
        black_text = FONT.render("Press B for Black", True, BLACK)
        self.window_surf.blit(white_text, (WINDOW_WIDTH//2 - white_text.get_width()//2, 300))
        self.window_surf.blit(black_text, (WINDOW_WIDTH//2 - black_text.get_width()//2, 350))
        
        pygame.display.flip()
        
        while True:
            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == KEYDOWN:
                    if event.key == K_w:
                        return WHITE, BLACK
                    elif event.key == K_b:
                        return BLACK, WHITE
            self.main_clock.tick(FPS)
    
    def get_valid_moves(self, col, row, is_human):
        """Get all valid moves for a piece including captures"""
        if self.main_grid[col][row] != (self.human_token if is_human else self.AI_token):
            return []
        
        directions = [(-1, -1), (-1, 0), (-1, 1),
                     (0, -1),           (0, 1),
                     (1, -1),  (1, 0),  (1, 1)]
        
        normal_moves = []
        capture_moves = []
        opponent = self.AI_token if is_human else self.human_token
        
        # Check for normal moves and captures
        for dc, dr in directions:
            new_col, new_row = col + dc, row + dr
            if 0 <= new_col < GRID_WIDTH and 0 <= new_row < GRID_HEIGHT:
                if self.main_grid[new_col][new_row] == EMPTY:
                    normal_moves.append((new_col, new_row))
                elif self.main_grid[new_col][new_row] == opponent:
                    # Check for approach capture (moving toward opponent)
                    approach_col, approach_row = new_col + dc, new_row + dr
                    if (0 <= approach_col < GRID_WIDTH and 0 <= approach_row < GRID_HEIGHT and
                        self.main_grid[approach_col][approach_row] == EMPTY):
                        capture_moves.append((approach_col, approach_row))
                    
                    # Check for withdrawal capture (moving away from opponent)
                    withdrawal_col, withdrawal_row = col - dc, row - dr
                    if (0 <= withdrawal_col < GRID_WIDTH and 0 <= withdrawal_row < GRID_HEIGHT and
                        self.main_grid[withdrawal_col][withdrawal_row] == EMPTY):
                        capture_moves.append((withdrawal_col, withdrawal_row))
        
        # Captures are mandatory
        return capture_moves if capture_moves else normal_moves
    
    def make_move(self, start, end, is_human):
        """Execute a move and handle captures"""
        start_col, start_row = start
        end_col, end_row = end
        moving_piece = self.main_grid[start_col][start_row]
        opponent = self.AI_token if is_human else self.human_token
        
        # Move the piece
        self.main_grid[end_col][end_row] = moving_piece
        self.main_grid[start_col][start_row] = EMPTY
        
        # Determine capture type and direction
        dc = end_col - start_col
        dr = end_row - start_row
        if dc != 0:
            dc = dc // abs(dc)
        if dr != 0:
            dr = dr // abs(dr)
        
        captured_pieces = []
        
        # Check for approach capture (moving toward opponent)
        approach_col, approach_row = start_col + dc, start_row + dr
        if (0 <= approach_col < GRID_WIDTH and 0 <= approach_row < GRID_HEIGHT and
            self.main_grid[approach_col][approach_row] == opponent):
            # Capture all consecutive opponent pieces in this direction
            while (0 <= approach_col < GRID_WIDTH and 0 <= approach_row < GRID_HEIGHT and
                   self.main_grid[approach_col][approach_row] == opponent):
                captured_pieces.append((approach_col, approach_row))
                approach_col += dc
                approach_row += dr
        
        # Check for withdrawal capture (moving away from opponent)
        withdrawal_col, withdrawal_row = start_col - dc, start_row - dr
        if (0 <= withdrawal_col < GRID_WIDTH and 0 <= withdrawal_row < GRID_HEIGHT and
            self.main_grid[withdrawal_col][withdrawal_row] == opponent):
            # Capture all consecutive opponent pieces in this direction
            while (0 <= withdrawal_col < GRID_WIDTH and 0 <= withdrawal_row < GRID_HEIGHT and
                   self.main_grid[withdrawal_col][withdrawal_row] == opponent):
                captured_pieces.append((withdrawal_col, withdrawal_row))
                withdrawal_col -= dc
                withdrawal_row -= dr
        
        # Remove captured pieces
        for cap_col, cap_row in captured_pieces:
            self.main_grid[cap_col][cap_row] = EMPTY
        
        return len(captured_pieces) > 0
    
    def check_win_condition(self):
        """Check if the game has been won"""
        human_pieces = sum(row.count(self.human_token) for row in self.main_grid)
        ai_pieces = sum(row.count(self.AI_token) for row in self.main_grid)
        
        if human_pieces == 0:
            self.winner = 'AI'
            return True
        elif ai_pieces == 0:
            self.winner = 'Human'
            return True
        
        # Check if current player has any valid moves
        current_token = self.human_token if self.turn == 'Human' else self.AI_token
        has_moves = False
        
        for col in range(GRID_WIDTH):
            for row in range(GRID_HEIGHT):
                if self.main_grid[col][row] == current_token:
                    if self.get_valid_moves(col, row, self.turn == 'Human'):
                        has_moves = True
                        break
            if has_moves:
                break
        
        if not has_moves:
            self.winner = 'AI' if self.turn == 'Human' else 'Human'
            return True
        
        return False
    
    def evaluate_board(self):
        """Evaluate the board state for the AI"""
        human_pieces = 0
        ai_pieces = 0
        
        for col in range(GRID_WIDTH):
            for row in range(GRID_HEIGHT):
                if self.main_grid[col][row] == self.human_token:
                    human_pieces += 1
                elif self.main_grid[col][row] == self.AI_token:
                    ai_pieces += 1
        
        if human_pieces == 0:
            return float('inf')  # AI wins
        elif ai_pieces == 0:
            return float('-inf')  # Human wins
        
        # Basic evaluation: piece difference with slight mobility consideration
        piece_diff = ai_pieces - human_pieces
        
        # Add small bonus for center control
        center_bonus = 0
        center_positions = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3), (3,1), (3,2), (3,3)]
        for col, row in center_positions:
            if self.main_grid[col][row] == self.AI_token:
                center_bonus += 0.1
            elif self.main_grid[col][row] == self.human_token:
                center_bonus -= 0.1
        
        return piece_diff + center_bonus
    
    def alpha_beta_search(self, depth):
        """Perform alpha-beta search to find best move"""
        self.AI_current_action = {}
        self.max_value(depth, -float('inf'), float('inf'), True)
        best_score = max(self.AI_current_action.keys())
        return self.AI_current_action[best_score]
    
    def max_value(self, depth, alpha, beta, is_root=False):
        """Maximizer part of minimax"""
        self.total_node_generated += 1
        self.depth_of_game_tree = max(self.depth_of_game_tree, depth)
        
        # Terminal test or depth limit
        if depth == 0 or self.check_win_condition():
            return self.evaluate_board()
        
        max_eval = -float('inf')
        best_move = None
        
        # Find all possible moves for AI
        for col in range(GRID_WIDTH):
            for row in range(GRID_HEIGHT):
                if self.main_grid[col][row] == self.AI_token:
                    valid_moves = self.get_valid_moves(col, row, False)
                    for move_col, move_row in valid_moves:
                        # Make a copy of the board and simulate the move
                        board_copy = copy.deepcopy(self.main_grid)
                        self.main_grid = board_copy
                        captured = self.make_move((col, row), (move_col, move_row), False)
                        
                        # If we made a capture, we get to move again
                        if captured:
                            eval = self.max_value(depth, alpha, beta)
                        else:
                            eval = self.min_value(depth - 1, alpha, beta)
                        
                        # Undo the move
                        self.main_grid = board_copy
                        
                        if eval > max_eval:
                            max_eval = eval
                            best_move = ((col, row), (move_col, move_row))
                            
                            if is_root:
                                self.AI_current_action[max_eval] = best_move
                            
                            alpha = max(alpha, eval)
                            if beta <= alpha:
                                self.pruning_in_max_value += 1
                                return max_eval
        
        return max_eval
    
    def min_value(self, depth, alpha, beta):
        """Minimizer part of minimax"""
        self.total_node_generated += 1
        self.depth_of_game_tree = max(self.depth_of_game_tree, depth)
        
        # Terminal test or depth limit
        if depth == 0 or self.check_win_condition():
            return self.evaluate_board()
        
        min_eval = float('inf')
        
        # Find all possible moves for human
        for col in range(GRID_WIDTH):
            for row in range(GRID_HEIGHT):
                if self.main_grid[col][row] == self.human_token:
                    valid_moves = self.get_valid_moves(col, row, True)
                    for move_col, move_row in valid_moves:
                        # Make a copy of the board and simulate the move
                        board_copy = copy.deepcopy(self.main_grid)
                        self.main_grid = board_copy
                        captured = self.make_move((col, row), (move_col, move_row), True)
                        
                        # If human made a capture, they move again
                        if captured:
                            eval = self.min_value(depth, alpha, beta)
                        else:
                            eval = self.max_value(depth - 1, alpha, beta)
                        
                        # Undo the move
                        self.main_grid = board_copy
                        
                        if eval < min_eval:
                            min_eval = eval
                            beta = min(beta, eval)
                            if beta <= alpha:
                                self.pruning_in_min_value += 1
                                return min_eval
        
        return min_eval
    
    def show_game_over(self):
        """Display game over message"""
        self.window_surf.fill(BROWN)
        
        if self.winner == 'Human':
            message = BIG_FONT.render("You Win!", True, GREEN)
        else:
            message = BIG_FONT.render("AI Wins!", True, RED)
        
        self.window_surf.blit(message, (WINDOW_WIDTH//2 - message.get_width()//2, WINDOW_HEIGHT//2))
        
        restart_text = FONT.render("Press R to restart or Q to quit", True, BLACK)
        self.window_surf.blit(restart_text, (WINDOW_WIDTH//2 - restart_text.get_width()//2, WINDOW_HEIGHT//2 + 50))
        
        pygame.display.flip()
        
        while True:
            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == KEYDOWN:
                    if event.key == K_r:
                        return True  # Restart
                    elif event.key == K_q:
                        pygame.quit()
                        sys.exit()
            self.main_clock.tick(FPS)
    
    def run(self):
        """Main game loop"""
        self.difficulty = self.show_difficulty_menu()
        self.human_token, self.AI_token = self.show_color_menu()
        self.turn = 'Human' if self.human_token == WHITE else 'AI'
        
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == QUIT:
                    running = False
                
                if event.type == MOUSEBUTTONDOWN and event.button == 1 and self.turn == 'Human':
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    col, row = self.get_grid_clicked(mouse_x, mouse_y)
                    
                    if col is not None and row is not None:
                        # If no piece selected yet, select one
                        if self.selected_piece is None:
                            if self.main_grid[col][row] == self.human_token:
                                self.selected_piece = (col, row)
                                self.valid_moves = self.get_valid_moves(col, row, True)
                        else:
                            # If clicked on a valid move, make the move
                            if (col, row) in self.valid_moves:
                                captured = self.make_move(self.selected_piece, (col, row), True)
                                self.selected_piece = None
                                self.valid_moves = []
                                
                                # If no capture was made, switch turns
                                if not captured:
                                    self.turn = 'AI'
                                
                                # Check win condition
                                if self.check_win_condition():
                                    self.draw_grid()
                                    if self.show_game_over():
                                        self.__init__()
                                        self.run()
                                        return
                            # If clicked on another of player's pieces, select that instead
                            elif self.main_grid[col][row] == self.human_token:
                                self.selected_piece = (col, row)
                                self.valid_moves = self.get_valid_moves(col, row, True)
            
            # AI's turn
            if self.turn == 'AI' and not self.game_over:
                # Reset AI stats
                self.AI_current_action = {}
                self.is_cutoff = False
                self.depth_of_game_tree = 0
                self.total_node_generated = 0
                self.pruning_in_max_value = 0
                self.pruning_in_min_value = 0
                
                # Make AI move based on difficulty
                if self.difficulty == '1':  # Random move
                    # Find all AI pieces with valid moves
                    ai_pieces = []
                    for col in range(GRID_WIDTH):
                        for row in range(GRID_HEIGHT):
                            if self.main_grid[col][row] == self.AI_token:
                                moves = self.get_valid_moves(col, row, False)
                                if moves:
                                    ai_pieces.append((col, row, moves))
                    
                    if ai_pieces:
                        # Choose random piece and random move
                        col, row, moves = random.choice(ai_pieces)
                        move_col, move_row = random.choice(moves)
                        captured = self.make_move((col, row), (move_col, move_row), False)
                        
                        # If no capture was made, switch turns
                        if not captured:
                            self.turn = 'Human'
                else:  # Use minimax
                    depth = 1 if self.difficulty == '2' else 3
                    (start_col, start_row), (end_col, end_row) = self.alpha_beta_search(depth)
                    captured = self.make_move((start_col, start_row), (end_col, end_row), False)
                    
                    # If no capture was made, switch turns
                    if not captured:
                        self.turn = 'Human'
                
                # Check win condition
                if self.check_win_condition():
                    self.draw_grid()
                    if self.show_game_over():
                        self.__init__()
                        self.run()
                        return
                
                # Small delay so human can see AI's move
                pygame.time.delay(500)
            
            self.draw_grid()
            self.main_clock.tick(FPS)
        
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = Fanorona()
    game.run()

SystemExit: 